In [4]:
import tensorflow as tf
import numpy as np

import os

In [2]:
def load_stl10_dataset(data_path, batch_size=32):
    with open(data_path, 'rb') as f:
        X = np.fromfile(f, dtype=np.uint8)
        X = np.reshape(X, (-1, 3, 96, 96))
        X = np.transpose(X, (0, 3, 2, 1))
        X = tf.data.Dataset.from_tensor_slices(X.astype(np.float32))
        return X.batch(batch_size)

In [3]:
DATA_FOLDER = "../data/stl10/stl10_binary/"
DATA_FILE   = "unlabeled_X.bin"

In [9]:
data_path = os.path.join(DATA_FOLDER, DATA_FILE)

tf_dataset = load_stl10_dataset(data_path, batch_size=32)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.